# 多変量での数式化と考察
機械学習のモデルを生成して結果精度が良かった場合そのモデルを採用できる。そこで、モデルによっては特徴量を変数として線形の式を出力でき、算出された係数から結果を与える条件が分かるようになる。しかし、この手法は非常に単純なデータで可能であり、様々な条件が絡み合うと適用できないこともある。

# ライブラリのインポート

In [ ]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.svm import LinearSVC as SVM
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import confusion_matrix,classification_report,mean_absolute_error
import pandas as pd
import numpy as np
import scipy.stats as stats

#カテゴリ分類で作られた決定境界の直線の数式を出力
判別問題であるため代表的なアルゴリズムとしてSVMを使用する。実際に分析をするときはマージンの大きさなどを調節して精度を良くすることで分析結果の正当性を上げることができる。

In [ ]:
df = pd.read_csv("iris.csv")
df.head()

,category,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,0,5.1,3.5,1.4,0.2
1,0,4.9,3.0,1.4,0.2
2,0,4.7,3.2,1.3,0.2
3,0,4.6,3.1,1.5,0.2
4,0,5.0,3.6,1.4,0.2


In [ ]:
y_name = "category"
y = df[y_name].values
x_table = df.drop([y_name], axis=1)
x_name = x_table.columns
x = x_table.values

SVMを使用する。実際にはマージンを調整することでより正確になる

In [ ]:
model = SVM()#マージンを調節するとより正確になる
model.fit(x, y)
y_pred = model.predict(x)

C:\Users\decar\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


混合行列を出力

In [ ]:
df_m = pd.DataFrame(confusion_matrix(y, y_pred))
df_m

,0,1,2
0,50,0,0
1,0,47,3
2,0,2,48


精度を出力する。この時、精度に不備がある場合算出された数式はあてにならない

In [ ]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.96      0.94      0.95        50
           2       0.94      0.96      0.95        50

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150



In [ ]:
fout = model.coef_
b = model.intercept_
b = b.reshape(-1, 1)
fout = np.hstack((fout, b))

クラス別分類直線を出力

In [ ]:
col = list(x_name)
col.append("intercept")
df_fout = pd.DataFrame(fout)
df_fout.columns = col
df_fout

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),intercept
0,0.184245,0.451227,-0.807937,-0.450713,0.109565
1,0.051699,-0.890806,0.404542,-0.937962,1.665632
2,-0.850713,-0.986574,1.381048,1.865291,-1.709658


生データでは尺度が違うため数値の影響を加味した数式を算出(0～1に正規化することで係数の大きい項目が影響力の強い項目と分かる)

In [ ]:
x = minmax_scale(x)
model.fit(x, y)
y_pred = model.predict(x)

In [ ]:
df_m = pd.DataFrame(confusion_matrix(y, y_pred))
df_m

,0,1,2
0,50,0,0
1,0,45,5
2,0,3,47


In [ ]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       0.94      0.90      0.92        50
           2       0.90      0.94      0.92        50

    accuracy                           0.95       150
   macro avg       0.95      0.95      0.95       150
weighted avg       0.95      0.95      0.95       150



In [ ]:
fout = model.coef_
b = model.intercept_
b = b.reshape(-1, 1)
fout = np.hstack((fout, b))

In [ ]:
col = list(x_name)
col.append("intercept")
df_fout = pd.DataFrame(fout)
df_fout.columns = col
df_fout

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),intercept
0,-0.612055,1.319489,-1.773791,-1.685438,0.588269
1,0.101019,-2.273601,1.227482,-1.167295,0.562984
2,-0.086573,-1.064235,2.261644,3.250393,-3.168260


判別問題では一定以上の値を取ると陽性(そうである)と判定され一定未満の値を取ると陰性(そうでない)と判定される。算出された係数が例えばマイナスなら数値が小さいと合計値が大きくなりプラスなら数値が大きいと合計値が大きくなる。

# 目的変数が数値である場合の数式化
ここでは代表的な統計学の手法で最小二乗法を用いる重回帰分析を使用する。

In [ ]:
df = pd.read_csv("boston.csv")
df.head()

,PRICE,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,21.6,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,34.7,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,33.4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,36.2,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


In [ ]:
y_name = "PRICE"
y = df[y_name].values
x_table = df.drop([y_name], axis=1)
x_name = x_table.columns
x = x_table.values

精度の算出(最も直感的な値として絶対誤差を使用)

In [ ]:
model = LR()
model.fit(x, y)
y_pred = model.predict(x)
print(mean_absolute_error(y, y_pred))

3.2708628109003155


相関係数で精度の確認と外れ値などの確認のため散布図を出力

In [ ]:
import matplotlib.pyplot as plt
pr = np.array([y_pred, y])
plt.scatter(y_pred, y)
plt.title("Correlation coefficient is %.2f"%(np.corrcoef(pr)[0][1]))
plt.xlabel("Predict")
plt.ylabel("Real")
plt.show()

<Figure size 640x480 with 1 Axes>

数式を作成

In [ ]:
fout = model.coef_
b = model.intercept_
fout = np.hstack((fout, b))

# 変数の影響度合いの算出
各変数のt値を算出することでp値を算出できる。p値が小さいほど変数の信用度が高くなる。逆に言えばp値が大きいと結果に寄与していない可能性があるため除外して考える必要性も出てくるため実際のデータ分析では分析に使う項目を選定してより精度の良いモデルを作成して分析することで正当性を示せる。

In [ ]:
sse = np.sum((y-y_pred)**2, axis=0)
sse = sse/(x.shape[0]-x. shape[1]-1)
s = np.linalg.inv(np.dot(x.T, x))
std_err = np.sqrt(np.diagonal(sse*s))
t = model.coef_ / std_err
p = stats.t(len(y)-len(x_name)-1).pdf(t)
dat =[fout, t, p]

In [ ]:
df_fout = pd.DataFrame(dat)
col = list(x_name)
col.append("intercept")
ind = ["fout", "t-value", "p-value"]
df_fout.columns = col
df_fout.index = ind
df_fout

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,intercept
fout,-0.108011,0.046420,0.020559,2.686734,-1.776661e+01,3.809865e+00,0.000692,-1.475567e+00,0.306049,-0.012335,-9.527472e-01,0.009312,-5.247584e-01,36.459488
t-value,-3.293348,3.382502,0.334836,3.119533,-5.551634e+00,1.293571e+01,0.052590,-7.916161e+00,4.811897,-3.299806,-9.101082e+00,3.624213,-1.084439e+01,NaN
p-value,0.001846,0.001379,0.376962,0.003191,1.247788e-07,1.823206e-32,0.398187,5.839671e-14,0.000005,0.001808,8.783676e-18,0.000603,4.532839e-24,NaN
